## **Processamento de Linguagem Natural (NLP)**

[Desafio Kaggle YouTube](https://www.kaggle.com/datasnaek/youtube-new)

O dataset do Youtube trends possui variáveis textuais contendo informações promissoras que podem ajudar os modelos regressores ou classificadores.

Entretanto, o texto precisa ser adequadamente limpo e processado. Empregaremos as técnicas de NLP e regex. 

Uma vez que o texto foi processado, aplicaremos o algoritmo de clusterização famoso em processamento textual, denominado Latent Dirichlet Allocation (LDA) sobre os títulos e as tags dos Youtube trend vídeos.

In [1]:
import pandas as pd
import numpy as np
from re import sub
import matplotlib.pyplot as plt
from numpy import asarray

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk import download

from gensim import corpora, models
import gensim

from warnings import filterwarnings

filterwarnings('ignore')
download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Sequência de tratamento do NLP**

In [0]:
# Geração dos tokens
tokenizer = RegexpTokenizer(r'\w+')

# Carrega da lista de palavras de pouco valor semântico da língua inglesa
en_stop = stopwords.words('english')

# Criação do objeto da classe PorterStemmer
porter_stem = PorterStemmer()

# Aplicando funções de NLP

def NLP_processing(text):
  token_list = []
  for k in text:
    # limpeza e tokenização
    temp = sub(r'[^\w\s]', ' ', k) 
    temp = sub(r'[^\D]', ' ', temp)     
    raw = temp.lower()
    tokens = tokenizer.tokenize(raw)   

    # remoção das stop words
    stop_tokens = [tok for tok in tokens if not tok in en_stop]

    #stematização dos tokens
    stemmed_tokens = [porter_stem.stem(j) for j in stop_tokens]     
    
    # adiciona tokens na lista
    token_list.append(stemmed_tokens)

  return token_list



In [3]:
# Carregando dataset pre-processado dos US
df = pd.read_csv('sentiments.csv')
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'description', 'likes_perc', 'dislikes_perc', 'comment_perc', 'emotion',
       'pop'],
      dtype='object')

In [4]:
df.shape

(6436, 17)

**Título**

Iremos investigar a feature title do dataset. A ideia é transformar cada título em um documento e processá-lo por NLP. 

Os termos tokenizados serão representados em uma outra dimensão de representação. Nesta dimensão altamente esparsa, agruparemos por LDA os documentos levando em conta ao mesmo tempo: 
- a frequência das palavras em cada documento 
- a proximidade dos documentos de acordo com essa frequência.


In [13]:
# Removendo registros sem texto
#text = df.title.dropna()
text = df.tags.dropna()
len(text)

6351

In [14]:
# Visualização de amostras de documentos tokenizados

token_list = NLP_processing(text)
token_list[:2]

[['ellen',
  'ellen',
  'degener',
  'ellen',
  'show',
  'ellentub',
  'ellen',
  'audienc',
  'season',
  'episod',
  'mindi',
  'kale',
  'mindi',
  'kale',
  'babi',
  'oprah',
  'mindi',
  'kale',
  'mindi',
  'kale',
  'offic',
  'mindi',
  'kale',
  'wrinkl',
  'time',
  'mindi',
  'kale',
  'b',
  'j',
  'novak',
  'katherin',
  'oprah',
  'hous',
  'ellen',
  'fan',
  'ellen',
  'ticket',
  'season',
  'daughter',
  'mindi',
  'kale',
  'daughter',
  'bj',
  'novak',
  'babi',
  'daddi',
  'ocean',
  'ocean',
  'movi',
  'offic',
  'interview',
  'new',
  'funni',
  'hilari',
  'sandra',
  'bullock',
  'ann',
  'hathaway',
  'wrinkl',
  'time'],
 ['megan',
  'mullal',
  'megan',
  'mullal',
  'grace',
  'karen',
  'grace',
  'actress',
  'nick',
  'offerman',
  'ellen',
  'degener',
  'ellen',
  'degener',
  'ellen',
  'show',
  'ellen',
  'fan',
  'ellen',
  'ticket',
  'ellentub',
  'ellen',
  'audienc',
  'grace',
  'karen',
  'roomat',
  'funni',
  'interview',
  'nick',
 

Uma vez que temos cada texto limpo e tratado por NLP, iremos aplicar um algoritmo de clusterização para identificar padrões nos textos.

In [0]:
# Transforma em um corpo de dicionário id 
dictionary = corpora.Dictionary(token_list)

In [0]:
# convert tokenized documents in document-term matrix
corpus = [dictionary.doc2bow(j) for j in token_list]

In [0]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20)

In [12]:
# parâmetros LDA
# num_topics: quantidade de tópicos (# clusters)
# num_words: quantidade de 

# o número na frente da palavra indica a porcentagem que ela contribui no cluster 
print(ldamodel.print_topics(num_topics=10, num_words = 4) )

[(0, '0.013*"american" + 0.012*"music" + 0.011*"vs" + 0.011*"full"'), (1, '0.013*"youtub" + 0.011*"react" + 0.010*"black" + 0.008*"movi"'), (2, '0.083*"offici" + 0.054*"video" + 0.042*"trailer" + 0.025*"hd"'), (3, '0.015*"live" + 0.009*"trailer" + 0.009*"beauti" + 0.009*"question"'), (4, '0.018*"makeup" + 0.012*"test" + 0.011*"work" + 0.010*"school"'), (5, '0.015*"make" + 0.009*"home" + 0.009*"j" + 0.008*"break"'), (6, '0.029*"first" + 0.014*"espn" + 0.012*"take" + 0.011*"time"'), (7, '0.008*"use" + 0.008*"stori" + 0.008*"falcon" + 0.008*"good"'), (8, '0.011*"battl" + 0.011*"grace" + 0.010*"made" + 0.009*"year"'), (9, '0.015*"super" + 0.014*"bowl" + 0.014*"star" + 0.013*"audio"')]


Tags

O mesmo procedimento é realizado para as tags dos vídeos do Youtube...

In [18]:
print(ldamodel.print_topics(num_topics=10, num_words = 4) )

[(0, '0.040*"news" + 0.011*"kardashian" + 0.010*"iphon" + 0.010*"none"'), (1, '0.019*"nba" + 0.016*"first" + 0.014*"game" + 0.014*"espn"'), (2, '0.027*"makeup" + 0.016*"food" + 0.013*"tutori" + 0.012*"challeng"'), (3, '0.014*"tour" + 0.012*"super" + 0.012*"news" + 0.011*"video"'), (4, '0.030*"music" + 0.025*"video" + 0.024*"cat" + 0.013*"song"'), (5, '0.035*"show" + 0.029*"funni" + 0.023*"late" + 0.020*"video"'), (6, '0.020*"anim" + 0.015*"star" + 0.013*"war" + 0.009*"smith"'), (7, '0.028*"ellen" + 0.013*"movi" + 0.009*"trailer" + 0.008*"degener"'), (8, '0.027*"dog" + 0.010*"christma" + 0.008*"video" + 0.008*"anim"'), (9, '0.023*"movi" + 0.021*"trailer" + 0.012*"netflix" + 0.009*"wwe"')]


#**Resultados**

Observamos alguns padrões interessantes tanto no processamento sobre o título quanto nas tags. Os clusters confirmam o mapa de palavras mais comuns apresentado na análise exploratória e contém palavras associadas às categorias mais comuns que são Entretenimento e Música e nos canais mais visualizados. 


*   **Título**

cluster: \["offici", "video", "trailer", "hd"] 

cluster: \["super", "bowl", "star", "audio"']

cluster: \["makeup", "test", "work", "school"'] 


*   **Tags**

cluster: \["ellen", "movi", "trailer", "degener"]

cluster: \["makeup", "food", "tutori", "challeng"]

cluster: \["nba", "first", "game", "espn"]


Algumas regras de negócios podem ser inferidas, por exemplo, sabe-se que vídeos de maquiagem são muito populares. 

Entretanto o cluster indica uma frequência significativa entre os vídeos trends daqueles com termos para reviews ("test") e para maquiagem de uso no trabalho ou para estudantes. 

